# Importing data from kaggle

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
!kaggle datasets download -d andrewmvd/face-mask-detection

 99% 395M/398M [00:19<00:00, 22.5MB/s]
100% 398M/398M [00:20<00:00, 20.8MB/s]


In [3]:
!unzip face-mask-detection.zip

Archive:  face-mask-detection.zip
  inflating: annotations/maksssksksss0.xml  
  inflating: annotations/maksssksksss1.xml  
  inflating: annotations/maksssksksss10.xml  
  inflating: annotations/maksssksksss100.xml  
  inflating: annotations/maksssksksss101.xml  
  inflating: annotations/maksssksksss102.xml  
  inflating: annotations/maksssksksss103.xml  
  inflating: annotations/maksssksksss104.xml  
  inflating: annotations/maksssksksss105.xml  
  inflating: annotations/maksssksksss106.xml  
  inflating: annotations/maksssksksss107.xml  
  inflating: annotations/maksssksksss108.xml  
  inflating: annotations/maksssksksss109.xml  
  inflating: annotations/maksssksksss11.xml  
  inflating: annotations/maksssksksss110.xml  
  inflating: annotations/maksssksksss111.xml  
  inflating: annotations/maksssksksss112.xml  
  inflating: annotations/maksssksksss113.xml  
  inflating: annotations/maksssksksss114.xml  
  inflating: annotations/maksssksksss115.xml  
  inflating: annotations/maksssk

In [4]:
!rm face-mask-detection.zip

# Data Preprocessing

## Converting XML Files to Text Files
## Splitting the Data into Train, Test and Validation

In [5]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import zipfile
from pathlib import Path
import numpy as np
import shutil

classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']
ROOT_DIR = "/content"
IMAGE_EXT = ".png"

DATA_DIR = ROOT_DIR
IMAGE_DIR = join(DATA_DIR, "images")
LABEL_DIR = join(DATA_DIR, "annotations")
PROCESSED_LABEL_DIR = join(DATA_DIR, "annotationstxt")
TRAIN_DATA_DIR = join(DATA_DIR, "train")
VALID_DATA_DIR = join(DATA_DIR, "valid")

Path(TRAIN_DATA_DIR).mkdir(parents=True, exist_ok=True)
Path(VALID_DATA_DIR).mkdir(parents=True, exist_ok=True)
Path(TRAIN_DATA_DIR + "/images").mkdir(parents=True, exist_ok=True)
Path(TRAIN_DATA_DIR + "/labels").mkdir(parents=True, exist_ok=True)
Path(VALID_DATA_DIR + "/images").mkdir(parents=True, exist_ok=True)
Path(VALID_DATA_DIR + "/labels").mkdir(parents=True, exist_ok=True)
Path(PROCESSED_LABEL_DIR).mkdir(parents=True, exist_ok=True)

# referred to https://github.com/pjreddie/darknet/blob/master/scripts/voc_label.py
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x,y,w,h)

# referred to https://github.com/pjreddie/darknet/blob/master/scripts/voc_label.py
def convert_annotation(classes, input_path, output_path):
    basename = os.path.basename(input_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(input_path)
    out_file = open(output_path + "/" + basename_no_ext + '.txt', 'w')

    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

    in_file.close()
    out_file.close()

paths = glob.glob(LABEL_DIR + '/*.xml')

for xml_path in tqdm(paths):
    convert_annotation(classes, xml_path, PROCESSED_LABEL_DIR)

label_files = glob.glob(PROCESSED_LABEL_DIR + '/*.txt')

train_indices, valid_indices = train_test_split(
    np.arange(len(label_files)), test_size=0.2, random_state=42, shuffle=True)

train_labels = []
for idx in train_indices:
    train_labels.append(label_files[idx])

valid_labels = []
for idx in valid_indices:
    valid_labels.append(label_files[idx])

for label_path in train_labels:
    basename = os.path.basename(label_path)
    basename_no_ext = os.path.splitext(basename)[0]

    shutil.move(label_path, join(TRAIN_DATA_DIR, "labels", basename_no_ext + ".txt"))
    shutil.move(join(IMAGE_DIR, basename_no_ext + IMAGE_EXT), join(TRAIN_DATA_DIR, "images", basename_no_ext + IMAGE_EXT))

for label_path in valid_labels:
    basename = os.path.basename(label_path)
    basename_no_ext = os.path.splitext(basename)[0]

    shutil.move(label_path, join(VALID_DATA_DIR, "labels", basename_no_ext + ".txt"))
    shutil.move(join(IMAGE_DIR, basename_no_ext + IMAGE_EXT), join(VALID_DATA_DIR, "images", basename_no_ext + IMAGE_EXT))

100%|██████████| 853/853 [00:00<00:00, 4899.06it/s]


In [6]:
# Finding labels

import re

def detect_first_letters_in_files(folder_path):
    with_mask = 0
    without_mask = 0
    mask_weared_incorrect = 0
    if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
        print("Invalid folder path.")
        return

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                for line in file:
                    first_letter = re.search(r'\b\w', line)
                    if first_letter:
                        if(first_letter.group()=='0'):
                            print(f"File: {filename}, First letter: {first_letter.group()}")
                            with_mask = with_mask+1
                        if(first_letter.group()=='1'):
                            print(f"File: {filename}, First letter: {first_letter.group()}")
                            without_mask = without_mask+1
                        if(first_letter.group()=='2'):
                            print(f"File: {filename}, First letter: {first_letter.group()}")
                            mask_weared_incorrect = mask_weared_incorrect+1

    print(f"No. of labels with mask: {with_mask}")
    print(f"No. of labels without mask: {without_mask}")
    print(f"No. of labels with mask wore incorrect: {mask_weared_incorrect}")


folder_path = '/content/train/labels'  # Replace with the actual path to your folder
detect_first_letters_in_files(folder_path)


File: maksssksksss95.txt, First letter: 1
File: maksssksksss421.txt, First letter: 0
File: maksssksksss421.txt, First letter: 0
File: maksssksksss421.txt, First letter: 0
File: maksssksksss421.txt, First letter: 0
File: maksssksksss421.txt, First letter: 0
File: maksssksksss593.txt, First letter: 0
File: maksssksksss593.txt, First letter: 0
File: maksssksksss593.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss573.txt, First letter: 0
File: maksssksksss790.txt, First letter: 0
File: maksssksksss10.txt, First letter: 0
File: maksssksksss699.txt, First letter: 1
File: maksssksksss699.txt, First letter: 1
File: maksssksksss699.txt, First letter: 0
File: maksssksksss699.txt, First letter: 1
File: maksssksksss699.txt, First letter: 1
File: maksssksksss699.txt, First letter: 1
File: maksssk

In [7]:
folder_path = '/content/valid/labels'  # Replace with the actual path to your folder
detect_first_letters_in_files(folder_path)

File: maksssksksss649.txt, First letter: 0
File: maksssksksss106.txt, First letter: 0
File: maksssksksss452.txt, First letter: 0
File: maksssksksss452.txt, First letter: 0
File: maksssksksss452.txt, First letter: 0
File: maksssksksss313.txt, First letter: 0
File: maksssksksss101.txt, First letter: 0
File: maksssksksss401.txt, First letter: 1
File: maksssksksss401.txt, First letter: 1
File: maksssksksss401.txt, First letter: 0
File: maksssksksss401.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 0
File: maksssksksss732.txt, First letter: 1
File: maksssksksss732.txt, First letter: 1
File: maksssksksss214.txt, First letter: 0
File: maksssksksss214.txt, First letter: 0
File: maksssksksss214.txt, First letter: 0
File: maksssksksss214.txt, First letter: 0
File: makss

In [8]:
# Count no of files
path,dir,files = next(os.walk('/content/train/images'))
train_count = len(files)
print(f"Number of images in train: {train_count}")

path,dir,files = next(os.walk('/content/valid/images'))
val_count = len(files)
print(f"Number of images in val: {val_count}")

path,dir,files = next(os.walk('/content/train/labels'))
train_count = len(files)
print(f"Number of labels in train: {train_count}")

path,dir,files = next(os.walk('/content/valid/labels'))
val_count = len(files)
print(f"Number of labels in val: {val_count}")

Number of images in train: 682
Number of images in val: 171
Number of labels in train: 682
Number of labels in val: 171


#Install YOLOV8

In [9]:
!pip install ultralytics

from IPython import display
display.clear_output()
!yolo checks

Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.6/78.2 GB disk)


In [10]:
from ultralytics import YOLO
from IPython.display import display,Image

# Training the dataset

## Labels

###   0   :    with_mask (WM)
###   1   :    without_mask (WoM)
###   2   :    mask_weared_incorrect (MWI)

In [11]:
!yolo task=detect mode=train model=yolov8n.pt data='/content/labels.yaml' epochs=200  imgsz=640

100% 6.23M/6.23M [00:00<00:00, 255MB/s]
Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/labels.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, kera

# Validating the trained model

In [12]:
!yolo task=detect mode=val model='/content/runs/detect/train/weights/best.pt' data='/content/labels.yaml'

Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients
val: Scanning /content/valid/labels.cache... 171 images, 0 backgrounds, 0 corrupt: 100% 171/171 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 11/11 [00:09<00:00,  1.21it/s]
                   all        171        828      0.807      0.772      0.829       0.56
                    WM        171        667      0.937       0.91      0.956      0.657
                   WoM        171        137      0.763      0.759      0.781      0.473
                   MWI        171         24      0.721      0.646      0.748      0.552
Speed: 0.9ms preprocess, 12.1ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/val


# Testing the trained model

In [13]:
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' conf=0.20 source='/content/valid/images'

Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients

image 1/171 /content/valid/images/maksssksksss101.png: 640x512 1 WM, 149.7ms
image 2/171 /content/valid/images/maksssksksss106.png: 448x640 (no detections), 49.7ms
image 3/171 /content/valid/images/maksssksksss118.png: 448x640 2 WMs, 6.0ms
image 4/171 /content/valid/images/maksssksksss120.png: 640x512 1 WM, 6.6ms
image 5/171 /content/valid/images/maksssksksss130.png: 640x544 3 WoMs, 1 MWI, 48.6ms
image 6/171 /content/valid/images/maksssksksss141.png: 448x640 2 WMs, 6.6ms
image 7/171 /content/valid/images/maksssksksss147.png: 416x640 3 WMs, 1 WoM, 49.0ms
image 8/171 /content/valid/images/maksssksksss149.png: 480x640 7 WMs, 1 WoM, 49.8ms
image 9/171 /content/valid/images/maksssksksss152.png: 480x640 7 WMs, 6.0ms
image 10/171 /content/valid/images/maksssksksss16.png: 640x512 1 WM, 7.8ms
image 11/171 /content/valid/images/maksssksksss16

In [14]:
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' conf=0.20 source='/content/custom_test'

Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients

image 1/12 /content/custom_test/20210906_180602.jpg: 640x480 2 WoMs, 58.8ms
image 2/12 /content/custom_test/20210906_181134.jpg: 640x480 1 WM, 1 MWI, 6.7ms
image 3/12 /content/custom_test/20210908_090150.jpg: 640x480 2 WoMs, 6.0ms
image 4/12 /content/custom_test/20210914_130247.jpg: 640x480 1 WoM, 6.0ms
image 5/12 /content/custom_test/20211006_164705.jpg: 640x480 1 WoM, 1 MWI, 6.0ms
image 6/12 /content/custom_test/20211014_064920.jpg: 640x480 2 WMs, 1 WoM, 1 MWI, 6.3ms
image 7/12 /content/custom_test/20211014_065011.jpg: 640x480 1 WoM, 1 MWI, 9.6ms
image 8/12 /content/custom_test/20211014_071505.jpg: 640x480 1 WoM, 7.6ms
image 9/12 /content/custom_test/20211014_072048.jpg: 480x640 2 WMs, 2 WoMs, 83.0ms
image 10/12 /content/custom_test/20211019_231335.jpg: 480x640 4 WoMs, 1 MWI, 7.6ms
image 11/12 /content/custom_test/20211019_231347.

# Exporting Model

In [15]:
!yolo mode=export model='/content/runs/detect/train/weights/best.pt' format=onnx

Ultralytics YOLOv8.0.154 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 289.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.7s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 7.6s, saved as '/content/runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (9

# Downloading Results

In [16]:
!zip -r runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/predict/ (stored 0%)
  adding: content/runs/detect/predict/maksssksksss386.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss739.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss486.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss664.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss141.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss791.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss301.png (deflated 1%)
  adding: content/runs/detect/predict/maksssksksss394.png (deflated 3%)
  adding: content/runs/detect/predict/maksssksksss256.png (deflated 2%)
  adding: content/runs/detect/predict/maksssksksss413.png (deflated 0%)
  adding: content/runs/detect/predict/maksssksksss168.png (deflated 0%)
  adding: content/runs/detect/predict/maksssksksss152.png (deflated 2%)
  addi

In [17]:
from google.colab import files
files.download('runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>